# 这里开始处理日文

## pykakasi自带分词和转换

In [5]:
from pykakasi import kakasi, wakati

In [4]:
kakasi_ins = kakasi()
kakasi_ins.setMode("H","a") # Hiragana to ascii, default: no conversion
kakasi_ins.setMode("K","a") # Katakana to ascii, default: no conversion
kakasi_ins.setMode("J","a") # Japanese to ascii, default: no conversion
kakasi_ins.setMode("r","Hepburn") # default: use Hepburn Roman table
kakasi_ins.setMode("s", True) # add space, default: no separator
# kakasi_ins.setMode("C", True) # capitalize, default: no capitalize
conv_pykakasi = kakasi_ins.getConverter()

wakati_ins = wakati()
sep_pykakasi = wakati_ins.getConverter()


def pykakasi_sep(text):
    text = text.replace("「","")
    text = text.replace("」","")
    text = sep_pykakasi.do(text)
    return text

def pykakasi_conv(text):
    text = text.replace("「","")
    text = text.replace("」","")
    text = conv_pykakasi.do(text)
    return text

## mecab分词+pykakasi转换

In [5]:
import romkan
import MeCab
from unidecode import unidecode
wakati_mecab = MeCab.Tagger("-Owakati")
# yomi = MeCab.Tagger("-Oyomi")
# chasen = MeCab.Tagger("-Ochasen")

kakasi_ins2 = kakasi()
kakasi_ins2.setMode("H","a") # Hiragana to ascii, default: no conversion
kakasi_ins2.setMode("K","a") # Katakana to ascii, default: no conversion
kakasi_ins2.setMode("J","a") # Japanese to ascii, default: no conversion
kakasi_ins2.setMode("r","Hepburn") # default: use Hepburn Roman table
# kakasi_ins2.setMode("C", True) # capitalize, default: no capitalize
conv_pykakasi_nosep = kakasi_ins2.getConverter()

def mecab_sep(text):
    text = text.replace("「","")
    text = text.replace("」","")
    text = wakati_mecab.parse(text)
    return text

def mecab_conv(text):
    text = text.replace("「","")
    text = text.replace("」","")
    text = conv_pykakasi_nosep.do(wakati_mecab.parse(text))
    return text

In [6]:
text = "待ち構えていたその患者はすぐ津田の背後に現われた。津田は早く退却しなければならなくなった。"
text = "きょうはとてもうれしい"
text = "彼女は非常に会社に貢献した。" 
text = "こんばんは"
target = "machikamae te i ta sono kanja wa sugu tsuda no haigo ni araware ta 。 tsuda wa hayaku taikyaku shi nakere ba nara naku nat ta 。"

In [7]:
text = "津田は自分の都合を善く考えてから日取をきめる事にして室外に出た。章、終り。"
target = "tsuda wa jibun no tsugo- wo yoku kangae te kara hi to wo kimeru koto ni shi te shitsugai ni de ta 。 akira 、 owari 。"
text = "「腸まで続いているとすると、癒りっこないんですか」「そんな事はありません」"
target = "cho- made tsuzui te iru to suru to 、 nao ri k ko nai n desu ka son'na koto wa ari mase n"

In [8]:
print(pykakasi_sep(text))
print(pykakasi_conv(text))

腸 まで 続い ているとすると、 癒 りっこないんですかそんな 事 はありません
chou made tsuzui teirutosuruto、 yu rikkonaindesukasonna koto haarimasen


In [9]:
print(mecab_sep(text))
print(mecab_conv(text))

腸 まで 続い て いる と する と 、 癒り っこ ない ん です か そんな 事 は あり ませ ん 

chou made tsuzui te iru to suru to 、 yuri kko nai n desu ka sonna koto ha ari mase n 



## 处理数据集

### 读取css10日文数据

In [7]:
def_sample_rate = 22050

In [8]:
import os
import pandas as pd
import sys
sys.path.append('..')
from utils.text import _phonemize

/opt/conda/lib/python3.6/site-packages/numba/errors.py:105: UserWarning: Insufficiently recent colorama version found. Numba requires colorama >= 0.3.9
  warnings.warn(msg)


In [4]:
txt_file = "css10/japanese/transcript.txt"
transcript_list = []
with open(txt_file, encoding='utf-8') as f:
    for line in f:
        parts = line.strip().split('|')
        transcript_list.append(parts)        

df = pd.DataFrame(transcript_list) #[order]

In [5]:
df

,0,1,2,3
0,meian/meian_0000.wav,この前探った時は、途中に瘢痕の隆起があったので、ついそこが行きどまりだとばかり思って、ああ云...,kono mae sagut ta toki wa 、 tochu- ni hankon n...,8.77
1,meian/meian_0001.wav,今日疎通を好くするために、そいつをがりがり掻き落して見ると、まだ奥があるんです」,kyo- sotsu- wo yoku suru tame ni 、 soitsu wo g...,7.48
2,meian/meian_0002.wav,津田の顔には苦笑の裡に淡く盛り上げられた失望の色が見えた。医者は白いだぶだぶした上着の前に,tsuda no kao ni wa kusho- no ura ni awaku mori...,9.11
3,meian/meian_0003.wav,両手を組み合わせたまま、ちょっと首を傾けた。その様子が「御気の毒ですが,ryo-te wo kumiawase ta mama 、 chotto kubi wo k...,7.44
4,meian/meian_0004.wav,津田は無言のまま帯を締め直して、椅子の背に投げ掛けられた袴を取り上げながらまた医者の方を向いた。,tsuda wa mugon no mama obi wo shime naoshi te ...,9.26
5,meian/meian_0005.wav,「腸まで続いているとすると、癒りっこないんですか」「そんな事はありません」,cho- made tsuzui te iru to suru to 、 nao ri k ...,5.7
6,meian/meian_0006.wav,医者は活溌にまた無雑作に津田の言葉を否定した。併せて彼の気分をも否定するごとくに。,isha wa kappatsu ni mata mu zo-sa ni tsuda no ...,9.09
7,meian/meian_0007.wav,「ただ今までのように穴の掃除ばかりしていては駄目なんです。それじゃいつまで経っても肉の上りこ...,tadaima made no yo- ni ana no so-ji bakari shi...,9.34
8,meian/meian_0008.wav,「切開です。切開して穴と腸といっしょにしてしまうんです。すると,sekkai desu 。 sekkai shi te ana to cho- to iss...,6.15
9,meian/meian_0009.wav,天然自然割かれた面の両側が癒着して来ますから、まあ本式に癒るようになるんです」,ten'nen shizen saka re ta men no ryo-gawa ga y...,7.68


In [8]:
txt_file = "css10/train_ja.txt"
txt_file_new = "css10/train_ja_new.txt"

In [6]:
txt_file = "css10/val_ja.txt"
txt_file_new = "css10/val_ja_new.txt"

In [9]:
textlist=[]
with open(txt_file, 'r', encoding='utf-8') as f:
    with open(txt_file_new, 'w', encoding='utf-8') as f2:
        
        for line in f:
            parts = line.strip().split('|')
    #         parts[1]="00-ja"
    #         parts[2]="ja"
    #         data_path = parts[3].replace("japanese/","")
    #         text_ori = df[(df[0]==data_path)].iloc[0][1]
    #         parts[3]= os.path.join("../css10/",parts[3])
    #         parts[4]= os.path.join("../css10/",parts[4])
    #         parts[5]= os.path.join("../css10/",parts[5])
    # #         text_roma = pykakasi_conv(text_ori)
            parts[7] = _phonemize(parts[6], 'ja')
            line_mod = '|'.join(parts)
            print(line_mod,file=f2)
#             textlist.append(line_mod)

print('Done')

[WARNING] 1 utterances containing language switches on lines 4
[WARNING] extra phones may appear in the "ja" phoneset
[WARNING] language switch flags have been removed (applying "remove-flags" policy)
[WARNING] 1 utterances containing language switches on lines 1
[WARNING] extra phones may appear in the "ja" phoneset
[WARNING] language switch flags have been removed (applying "remove-flags" policy)
[WARNING] 1 utterances containing language switches on lines 3
[WARNING] extra phones may appear in the "ja" phoneset
[WARNING] language switch flags have been removed (applying "remove-flags" policy)
[WARNING] 2 utterances containing language switches on lines 1, 2
[WARNING] extra phones may appear in the "ja" phoneset
[WARNING] language switch flags have been removed (applying "remove-flags" policy)
[WARNING] 1 utterances containing language switches on lines 3
[WARNING] extra phones may appear in the "ja" phoneset
[WARNING] language switch flags have been removed (applying "remove-flags" 

Done


### 读取Mozilla日文数据

In [10]:
import glob, os
from pydub import AudioSegment

In [11]:
txt_file = "Mozilla/ja/train.tsv"

In [12]:
df = pd.read_csv(txt_file,sep='\t')
df

,client_id,path,sentence,up_votes,down_votes,age,gender,accent,locale,segment
0,11e6ed343ee4fcf30f8717b698eca87944399ff5e73b32...,common_voice_ja_20720467.mp3,本屋で旅行の本を買いました。,2,0,twenties,male,NaN,ja,NaN
1,11e6ed343ee4fcf30f8717b698eca87944399ff5e73b32...,common_voice_ja_20720591.mp3,この花は春になると、咲きます。,2,0,twenties,male,NaN,ja,NaN
2,11e6ed343ee4fcf30f8717b698eca87944399ff5e73b32...,common_voice_ja_20720589.mp3,彼女は夜遅くまで起きているのに慣れている。,2,1,twenties,male,NaN,ja,NaN
3,11e6ed343ee4fcf30f8717b698eca87944399ff5e73b32...,common_voice_ja_20720583.mp3,木村さんはとても頭がいいです。,2,0,twenties,male,NaN,ja,NaN
4,11e6ed343ee4fcf30f8717b698eca87944399ff5e73b32...,common_voice_ja_20720829.mp3,彼女は服を片付けた。,2,0,twenties,male,NaN,ja,NaN
5,11e6ed343ee4fcf30f8717b698eca87944399ff5e73b32...,common_voice_ja_20720831.mp3,彼女は非常に会社に貢献した。,2,0,twenties,male,NaN,ja,NaN
6,11e6ed343ee4fcf30f8717b698eca87944399ff5e73b32...,common_voice_ja_20720860.mp3,林さんの家には犬と猫がいます。,2,0,twenties,male,NaN,ja,NaN
7,11e6ed343ee4fcf30f8717b698eca87944399ff5e73b32...,common_voice_ja_20720865.mp3,パンや卵などを食べます。,2,0,twenties,male,NaN,ja,NaN
8,11e6ed343ee4fcf30f8717b698eca87944399ff5e73b32...,common_voice_ja_20720866.mp3,彼女は老人に親切です。,2,1,twenties,male,NaN,ja,NaN
9,11e6ed343ee4fcf30f8717b698eca87944399ff5e73b32...,common_voice_ja_20720967.mp3,サッカーより野球のほうが好きです。,2,0,twenties,male,NaN,ja,NaN


In [13]:
txt_file_list_all = glob.glob("Mozilla/ja/*.tsv")
txt_file_list_all

['Mozilla/ja/validated.tsv',
 'Mozilla/ja/reported.tsv',
 'Mozilla/ja/dev.tsv',
 'Mozilla/ja/train.tsv',
 'Mozilla/ja/invalidated.tsv',
 'Mozilla/ja/test.tsv',
 'Mozilla/ja/other.tsv']

In [14]:
txt_file_list_all.remove('Mozilla/ja/test.tsv')
txt_file_list_all.remove('Mozilla/ja/reported.tsv')

txt_file_list = txt_file_list_all
txt_file_list 

['Mozilla/ja/validated.tsv',
 'Mozilla/ja/dev.tsv',
 'Mozilla/ja/train.tsv',
 'Mozilla/ja/invalidated.tsv',
 'Mozilla/ja/other.tsv']

In [15]:
df_list = []
for txt_file in txt_file_list:
    df = pd.read_csv(txt_file,sep='\t')
    df_list.append(df)
df_sum = pd.concat(df_list)

In [16]:
df.loc[:,'up_votes'].value_counts()

0    725
Name: up_votes, dtype: int64

In [17]:
df.loc[:,'down_votes'].value_counts()

0    725
Name: down_votes, dtype: int64

In [18]:
df_sum.loc[:,'up_votes'].value_counts()

2     8762
0     1955
1      164
8      102
7       85
3       34
9       22
6        9
5        2
10       2
4        1
Name: up_votes, dtype: int64

In [19]:
df_sum.loc[:,'down_votes'].value_counts()

0     9286
2     1391
1      455
3        5
10       1
Name: down_votes, dtype: int64

In [20]:
df_sum.loc[:,'client_id'].value_counts()

11e6ed343ee4fcf30f8717b698eca87944399ff5e73b32d65a5d568adefd7f996c9051a562dd8d4faf14ebd9b46faba3b3b4104b4fd974b7b95627c598e4b68d    1593
8e5df296063d264ffe317c0ac7828f1ed72e44a975edb144a42cc873b67cd6718530fff0a595cd07ab437fdd8164f840287e987d5bf4f05dcba9fad6274267ac    1289
51e9ed35447037faf8743d586cdb3e1ec9f5c36145456d83cad5e923288fd71278ff6fc1d932eb62a8693da3a224ac54418f66c8577dfa57bbc8a4f3497fe700    1221
b067e4a64d0c78c7c24b8eb93f9efc165121f9281fa6c31386d872529c2951a5a1f144ee8e5679c1bd41003695583b1c341de7d67fea995d584b5724b91ce984     656
60dd57f5407b98da0fa6c22506c3ccd7ad4e57ef949808d385b447f0e2b210f1c546ed55a6877145c95d807178c13df7577dbaac55015e60d76273cb2a715ea5     589
c26b5026d43709b6b49e3d5ef6a8805d0bf166e3d2f64b47581e24e9ea1a79cb066da370916090a4c758b4723d8d2fbee2be7e5760cf23bf7e5f13e03c6bac4f     450
b7163b95f7dfc60b56cea7b0f7e9ab23f80b52635b29826bf868c1a3b84f752c440f455bf3b856f68a6dab3d3d64e2de81889c68d2401c6cdca24db1b117347f     376
ae366c54d8ae632ccd1098f7d9462913e0b7688c1

In [21]:
df

,client_id,path,sentence,up_votes,down_votes,age,gender,accent,locale,segment
0,da263649c681c6c5a50a5dc6252c3cc394baa2246f0c72...,common_voice_ja_22384698.mp3,このタイプの人間が、一番多量の仕事をする。,0,0,twenties,female,NaN,ja,NaN
1,da263649c681c6c5a50a5dc6252c3cc394baa2246f0c72...,common_voice_ja_22384696.mp3,部屋を替えていただけますか。,0,0,twenties,female,NaN,ja,NaN
2,5ccd151b6c8adb14452960ab0763ce25872fa24b8dedbc...,common_voice_ja_22392991.mp3,いつも朝八時の電車に乗ります。,0,0,twenties,female,NaN,ja,NaN
3,5ccd151b6c8adb14452960ab0763ce25872fa24b8dedbc...,common_voice_ja_22392986.mp3,山田さんは目が悪いので、眼鏡をかけています。,0,0,twenties,female,NaN,ja,NaN
4,5ccd151b6c8adb14452960ab0763ce25872fa24b8dedbc...,common_voice_ja_22392985.mp3,あの会社は車を作っています。,0,0,twenties,female,NaN,ja,NaN
5,5ccd151b6c8adb14452960ab0763ce25872fa24b8dedbc...,common_voice_ja_22392989.mp3,膝の上にナプキンを広げなさい。,0,0,twenties,female,NaN,ja,NaN
6,5ccd151b6c8adb14452960ab0763ce25872fa24b8dedbc...,common_voice_ja_22392988.mp3,必ず辞書を自分の横に置いて起きなさい。,0,0,twenties,female,NaN,ja,NaN
7,5ccd151b6c8adb14452960ab0763ce25872fa24b8dedbc...,common_voice_ja_22392987.mp3,不思議なことに、結局彼は本当に試験に合格した。,0,0,twenties,female,NaN,ja,NaN
8,5ccd151b6c8adb14452960ab0763ce25872fa24b8dedbc...,common_voice_ja_22392992.mp3,平和と安全を切望しない人はいない。,0,0,twenties,female,NaN,ja,NaN
9,5ccd151b6c8adb14452960ab0763ce25872fa24b8dedbc...,common_voice_ja_22392994.mp3,部屋にはたくさんの少女がいる。,0,0,twenties,female,NaN,ja,NaN


In [22]:
df_sum

,client_id,path,sentence,up_votes,down_votes,age,gender,accent,locale,segment
0,087edae49ce1e0f600682ceccc7fc28e81e64ae890e647...,common_voice_ja_22072759.mp3,財布をなくしたので、交番へ行きます。,2,0,teens,female,NaN,ja,NaN
1,15b7d87a73d28b37664fdf7fea1ff232f89e80ce954c9b...,common_voice_ja_19499629.mp3,新しい靴をはいて出かけます。,2,0,NaN,NaN,NaN,ja,NaN
2,425f5a46e995ededc3a866f25ac1b5adba06f4ae991d1e...,common_voice_ja_19984402.mp3,必ずやります。,2,0,twenties,male,NaN,ja,NaN
3,4d037df65974988a11e23c8b1a853f70d3f02607bc3a9a...,common_voice_ja_21358328.mp3,妹がまだ帰らないので、母は心配しています。,2,0,twenties,male,NaN,ja,NaN
4,6c2b1130c2bc95dd93ee9edca0f4550951e14e09123bd0...,common_voice_ja_20946865.mp3,風が強くなりさえしなければいいですよ！,2,0,thirties,male,NaN,ja,NaN
5,6ef3526cdbe5773c8bcfeea7200469067786e3940a5552...,common_voice_ja_22219505.mp3,はち,2,0,NaN,NaN,NaN,ja,Singleword Benchmark
6,84b2e497ddfcc3cd7e7c9652cb9d673b3e109c0b79be1e...,common_voice_ja_20650478.mp3,外国語を勉強するのは難しいです。,2,0,NaN,NaN,NaN,ja,NaN
7,8d15a5cf215278a9869edb9801b26ef35a7e588077d2b2...,common_voice_ja_22307742.mp3,れい,2,0,NaN,NaN,NaN,ja,Singleword Benchmark
8,c1550c98cd768e824a7394ede788232a3665e4fd5a0fd5...,common_voice_ja_21493990.mp3,いろいろありがとうございました。,2,1,NaN,NaN,NaN,ja,NaN
9,dc4c7cb2b4b1baac5d54235744d1f7a707ea147492120c...,common_voice_ja_19676248.mp3,ぶらぶらと、球場まで十分足らずの道を歩いていく。,2,0,twenties,female,NaN,ja,NaN


In [23]:
output_metafile_name = "ja_Mozilla.txt"
metafile_path = os.path.join("Mozilla", output_metafile_name)
wav_dir = "Mozilla/ja/clips/"
resample_dir = "Mozilla/ja/clips_22k/"
sub_dir = "ja/clips_22k/"
if not os.path.exists(resample_dir): os.makedirs(resample_dir)
with open(metafile_path, 'w', encoding='utf-8') as f:

    for index, row in df_sum.iterrows(): 
        speaker = row.client_id[:5]
        language = "ja"
        audio_path = os.path.join(wav_dir, row.path)
        resample_path = os.path.join(resample_dir, os.path.splitext(row.path)[0]+".wav")
        if not os.path.exists(resample_path):
            sound = AudioSegment.from_file(audio_path)
            sound_resample = sound.set_frame_rate(def_sample_rate).set_channels(1)
            sound_resample.export(resample_path, format="wav")
        audio_path = os.path.join(sub_dir, os.path.splitext(row.path)[0]+".wav")
        spectrogram_paths = "|"
#         raw_text = pykakasi_conv(row.sentence)
        phonemized_text= _phonemize(raw_text,'ja')
        print(f'{str(index).zfill(6)}|{speaker}|{language}|{audio_path}|{spectrogram_paths}|{raw_text}|{phonemized_text}', file=f)
print('Done')

[WARNING] 1 utterances containing language switches on lines 2
[WARNING] extra phones may appear in the "ja" phoneset
[WARNING] language switch flags have been removed (applying "remove-flags" policy)
[WARNING] 1 utterances containing language switches on lines 1
[WARNING] extra phones may appear in the "ja" phoneset
[WARNING] language switch flags have been removed (applying "remove-flags" policy)
[WARNING] 1 utterances containing language switches on lines 2
[WARNING] extra phones may appear in the "ja" phoneset
[WARNING] language switch flags have been removed (applying "remove-flags" policy)
[WARNING] 1 utterances containing language switches on lines 1
[WARNING] extra phones may appear in the "ja" phoneset
[WARNING] language switch flags have been removed (applying "remove-flags" policy)
[WARNING] 1 utterances containing language switches on lines 1
[WARNING] extra phones may appear in the "ja" phoneset
[WARNING] language switch flags have been removed (applying "remove-flags" pol

[WARNING] 1 utterances containing language switches on lines 1
[WARNING] extra phones may appear in the "ja" phoneset
[WARNING] language switch flags have been removed (applying "remove-flags" policy)
[WARNING] 1 utterances containing language switches on lines 1
[WARNING] extra phones may appear in the "ja" phoneset
[WARNING] language switch flags have been removed (applying "remove-flags" policy)
[WARNING] 1 utterances containing language switches on lines 1
[WARNING] extra phones may appear in the "ja" phoneset
[WARNING] language switch flags have been removed (applying "remove-flags" policy)
[WARNING] 1 utterances containing language switches on lines 1
[WARNING] extra phones may appear in the "ja" phoneset
[WARNING] language switch flags have been removed (applying "remove-flags" policy)
[WARNING] 1 utterances containing language switches on lines 1
[WARNING] extra phones may appear in the "ja" phoneset
[WARNING] language switch flags have been removed (applying "remove-flags" pol

[WARNING] 1 utterances containing language switches on lines 1
[WARNING] extra phones may appear in the "ja" phoneset
[WARNING] language switch flags have been removed (applying "remove-flags" policy)
[WARNING] 1 utterances containing language switches on lines 2
[WARNING] extra phones may appear in the "ja" phoneset
[WARNING] language switch flags have been removed (applying "remove-flags" policy)
[WARNING] 1 utterances containing language switches on lines 1
[WARNING] extra phones may appear in the "ja" phoneset
[WARNING] language switch flags have been removed (applying "remove-flags" policy)
[WARNING] 1 utterances containing language switches on lines 1
[WARNING] extra phones may appear in the "ja" phoneset
[WARNING] language switch flags have been removed (applying "remove-flags" policy)
[WARNING] 1 utterances containing language switches on lines 1
[WARNING] extra phones may appear in the "ja" phoneset
[WARNING] language switch flags have been removed (applying "remove-flags" pol

[WARNING] 1 utterances containing language switches on lines 2
[WARNING] extra phones may appear in the "ja" phoneset
[WARNING] language switch flags have been removed (applying "remove-flags" policy)
[WARNING] 1 utterances containing language switches on lines 1
[WARNING] extra phones may appear in the "ja" phoneset
[WARNING] language switch flags have been removed (applying "remove-flags" policy)
[WARNING] 1 utterances containing language switches on lines 1
[WARNING] extra phones may appear in the "ja" phoneset
[WARNING] language switch flags have been removed (applying "remove-flags" policy)
[WARNING] 1 utterances containing language switches on lines 1
[WARNING] extra phones may appear in the "ja" phoneset
[WARNING] language switch flags have been removed (applying "remove-flags" policy)
[WARNING] 1 utterances containing language switches on lines 1
[WARNING] extra phones may appear in the "ja" phoneset
[WARNING] language switch flags have been removed (applying "remove-flags" pol

[WARNING] 2 utterances containing language switches on lines 1, 2
[WARNING] extra phones may appear in the "ja" phoneset
[WARNING] language switch flags have been removed (applying "remove-flags" policy)
[WARNING] 1 utterances containing language switches on lines 2
[WARNING] extra phones may appear in the "ja" phoneset
[WARNING] language switch flags have been removed (applying "remove-flags" policy)
[WARNING] 1 utterances containing language switches on lines 1
[WARNING] extra phones may appear in the "ja" phoneset
[WARNING] language switch flags have been removed (applying "remove-flags" policy)
[WARNING] 1 utterances containing language switches on lines 2
[WARNING] extra phones may appear in the "ja" phoneset
[WARNING] language switch flags have been removed (applying "remove-flags" policy)
[WARNING] 1 utterances containing language switches on lines 1
[WARNING] extra phones may appear in the "ja" phoneset
[WARNING] language switch flags have been removed (applying "remove-flags" 

Done


#  这里开始处理中文

In [2]:
import pandas as pd
import glob,os
import sys
sys.path.append('..')
from utils.text import to_phoneme,_phonemize
from utils.text import build_phoneme_dicts

In [3]:
from pypinyin import lazy_pinyin, Style, slug
import re, jieba
import pinyin
import jieba

In [4]:
def to_pinyin(text, punctuation="!:;,-.?，。！？｡：；、"):
    re_punctuation =r'\s+([{}])'.format(punctuation)
    tt=[]
    for tx in jieba.cut(text):
        tt.append(slug(tx, style= Style.TONE, separator=''))
    te = ' '.join(tt)
    text = re.sub(re_punctuation, r'\1', te)
    return text

def to_pinyin_pre(text):
    return ' '.join([pinyin.get(x) for x in jieba.cut(text.rstrip())])

In [5]:
s="上期主要讲了人类的直立行走，一个因自然环境变化。"
# s = "今天很高兴。"
print(to_pinyin_pre(s))
print(to_pinyin(s))

shàngqī zhǔyào jiǎng le rénlèi de zhílì xíngzǒu ， yīgè yīn zìránhuánjìng biànhuà 。
shàngqī zhǔyào jiǎng le rénlèi de zhílì xíngzǒu， yígè yīn zìránhuánjìng biànhuà。


In [6]:
_phonemize(to_pinyin(s),'cmn')

's.ɑ ŋtɕhi ts.u jiɑ ou5 tɕiɑ ŋ lə ʐo nlə ai5 tə ts.i. li ɕi ŋtso jiou5 ， ji ko ji ə5n tsi̪ ʐɑ nxwɑ ntɕi ŋ piɑ nxwɑ 。'

In [14]:
textlist=[]
with open('./re_gesw_6lang_22k_pinyin/train.txt') as f:
    for data in f.readlines():
        parts = data.strip().split('|')
        if parts[1]=='00-zh':
            parts[7]=_phonemize(to_pinyin(parts[6]),'cmn')
            data = '|'.join(parts)+"\n"
            
        textlist.append(data)
        
new_txt_file = 'train.txt'

with open(new_txt_file, 'w', encoding='utf-8') as f:
    for text in textlist:
        f.write(text)

[WARNING] 1 utterances containing language switches on lines 5
[WARNING] extra phones may appear in the "cmn" phoneset
[WARNING] language switch flags have been removed (applying "remove-flags" policy)
[WARNING] 1 utterances containing language switches on lines 5
[WARNING] extra phones may appear in the "cmn" phoneset
[WARNING] language switch flags have been removed (applying "remove-flags" policy)
[WARNING] 1 utterances containing language switches on lines 1
[WARNING] extra phones may appear in the "cmn" phoneset
[WARNING] language switch flags have been removed (applying "remove-flags" policy)
[WARNING] 1 utterances containing language switches on lines 3
[WARNING] extra phones may appear in the "cmn" phoneset
[WARNING] language switch flags have been removed (applying "remove-flags" policy)
[WARNING] 1 utterances containing language switches on lines 4
[WARNING] extra phones may appear in the "cmn" phoneset
[WARNING] language switch flags have been removed (applying "remove-flags

In [10]:
cmn_data=[
    {
        'name_dir':"Lance_Chn_CleanNoise_22k",
        'sub_dir':"Lance_Chn_CleanNoise",
        'trim_dir':"Lance_Chn_CleanNoise_22k_trim33",
        'output_metafile_name' : "Lance_Chn_CleanNoise_22k_trim33.txt",
        'speaker':"lance-zh"
    },
    {      
        'name_dir':"gg_Chn_CleanNoise_22k",
        'sub_dir': "gg_Chn_CleanNoise",
        'trim_dir':"gg_Chn_CleanNoise22k_trim33",
        'output_metafile_name':"gg_Chn_CleanNoise_22k_trim33.txt",
        'speaker':"gg-zh"
    },
    {
        'name_dir' : "cary_Chn",
        'sub_dir' : "cary_22k",
        'trim_dir' : "cary_22k_trim33",
        'output_metafile_name' : "cary_22k_trim33.txt",
        'speaker' : "cary-zh"
    },
    {
        'name_dir' : "Hotpot_Chn",
        'sub_dir' : "Hotpot_Chn",
        'trim_dir' : "Hotpot_Chn_22k_trim33",
        'output_metafile_name' : "Hotpot_Chn_22k_trim33.txt",
        'speaker' : "hotpot-zh"    
    },
    {
        'name_dir' : "thchs30_22k",
        'sub_dir' : "wav",
        'trim_dir' : "thchs30_22k_trim33",
        'output_metafile_name' : "thchs30_22k_trim33.txt",
        'speaker':""       
    }]
# resample = True
language = "zh"

In [11]:
for data in cmn_data:
    print(data['name_dir'])

Lance_Chn_CleanNoise_22k
gg_Chn_CleanNoise_22k
cary_Chn
Hotpot_Chn
thchs30_22k


In [12]:
xlsx_list=[]
for data in cmn_data:
    xlsx_list.append(glob.glob("./{}/*.xlsx".format(data['name_dir']))[0])
xlsx_list

['./Lance_Chn_CleanNoise_22k/2Voice-Chn-Lance_CleanNoise22k.xlsx',
 './gg_Chn_CleanNoise_22k/2Voice-Chn-gg_CleanNoise22k.xlsx',
 './cary_Chn/2Voice-Chn_cary_22k.xlsx',
 './Hotpot_Chn/2Voice-Chn_Hotpot_annv1.xlsx',
 './thchs30_22k/2Voice-Chn_thchs30_v2.xlsx']

In [13]:
# xlsx=xlsx_list[0]

In [14]:
df_list = []
for xlsx in xlsx_list:
    df = pd.read_excel(xlsx)
    df_list.append(df)
df_sum = pd.concat(df_list)

In [15]:
df_sum

,dataset_name,script_title,script_name,sentence_number,sentence_text,language,speaker,emotion,data_path,data_name,...,voice_format,voice_sr,prep_date_time,prep_path,prep_name,samples,frames,clean_date_time,cleaner_name,clean_text
0,Lance,大脑进化,JPHC-001,0,上期主要讲了人类的直立行走，一个因自然环境变化，而被迫完成的残酷过程。,Chn,Lance,一般,\Lance_Chn_CleanNoise\CleanNoise-987_0064_2.wav,CleanNoise-987_0064_2.wav,...,wav,22050,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Lance,大脑进化,JPHC-001,1,总结一下，树上的生活，虽然相对无忧无虑，但是为了采摘食物的方便，古猿还是有了站立能力。,Chn,Lance,一般,\Lance_Chn_CleanNoise\CleanNoise-987_0064_2.wav,CleanNoise-987_0064_2.wav,...,wav,22050,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Lance,大脑进化,JPHC-001,2,这就给后来树下的直立行走打下了基础。,Chn,Lance,一般,\Lance_Chn_CleanNoise\CleanNoise-987_0064_2.wav,CleanNoise-987_0064_2.wav,...,wav,22050,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Lance,大脑进化,JPHC-001,3,而为了升高姿势在草原上获得视野，去搜索目标和预警，是增强直立行走和奔跑能力的原因。,Chn,Lance,一般,\Lance_Chn_CleanNoise\CleanNoise-987_0064_2.wav,CleanNoise-987_0064_2.wav,...,wav,22050,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Lance,大脑进化,JPHC-001,4,不过两条腿的直立行走，却意外获得了能量优势。,Chn,Lance,一般,\Lance_Chn_CleanNoise\CleanNoise-987_0064_2.wav,CleanNoise-987_0064_2.wav,...,wav,22050,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13383,thchs30,NaN,NaN,NaN,杭城某丝织厂一位沈姓姑娘怀揣一千多元钱来到濮院选购羊毛衫。,Chn,D8,NaN,NaN,NaN,...,wav,22050,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13384,thchs30,NaN,NaN,NaN,他认为：无论是脑血栓还是脑溢血均为颅内血液循环障碍、脑组织受损所致。,Chn,D8,NaN,NaN,NaN,...,wav,22050,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13385,thchs30,NaN,NaN,NaN,非洲葡语国家包括安哥拉、莫桑比克、几内亚比绍、佛得角、圣多美和普林西比五国。,Chn,D8,NaN,NaN,NaN,...,wav,22050,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13386,thchs30,NaN,NaN,NaN,它的外形特征是头中等大、鼻梁多微隆、眼大明亮、耳较短厚、颈细、前胸宽广、蹄小而质坚。,Chn,D8,NaN,NaN,NaN,...,wav,22050,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
# Audio trim process. NO NEED TO EXECUTE EVERYTIME.
import sys
sys.path.append("../")
from utils import audio

stft_window_ms = 50                  # size in ms of the Hann window of short-time Fourier transform, used during spectrogram computation
stft_shift_ms = 12.5 

if not os.path.exists(os.path.join(name_dir, trim_dir)): os.makedirs(os.path.join(name_dir, trim_dir))
for index, row in df.iterrows(): 
    audio_path = os.path.join(name_dir, sub_dir, row.voice_name)
#     if resample:
#         sound = AudioSegment.from_file(audio_path)
#         sound_resample = sound.set_frame_rate(def_sample_rate).set_channels(1)
#         sound_resample.export(audio_path, format="wav")
    wav_data = audio.load(audio_path)
    wav_data_trim = audio.trim_silence(wav_data,stft_window_ms,stft_shift_ms,top_db=33)[0]
    audio_path_trim = os.path.join(name_dir, trim_dir, row.voice_name)
    if not os.path.exists(audio_path_trim):
        audio.save(wav_data_trim, audio_path_trim)

In [11]:
# text_lang_pairs=[(row.sentence_text,'cmn')for _,row in df_sum.iterrows()]
# phoneme_dict = build_phoneme_dicts(text_lang_pairs)

Building phoneme dictionary: ░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░ 0.0%

[ERROR] espeak: No envelope
[ERROR] espeak: No envelope
[ERROR] espeak: No envelope


Building phoneme dictionary: ░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░ 0.0%

[ERROR] espeak: No envelope
[ERROR] espeak: No envelope
[ERROR] espeak: No envelope
[ERROR] espeak: No envelope
[WARNING] 1 utterances containing language switches on lines 1
[WARNING] extra phones may appear in the "cmn" phoneset
[WARNING] language switch flags have been removed (applying "remove-flags" policy)


Building phoneme dictionary: ░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░ 0.1%

...

Building phoneme dictionary: ███████████████████████████████████████████████████████████▓░░░░░░░░░░░ 85.6%

[ERROR] espeak: No envelope
[ERROR] espeak: No envelope
[ERROR] espeak: No envelope
[ERROR] espeak: No envelope


Building phoneme dictionary: ███████████████████████████████████████████████████████████▓░░░░░░░░░░░ 85.6%

[WARNING] 1 utterances containing language switches on lines 1
[WARNING] extra phones may appear in the "cmn" phoneset
[WARNING] language switch flags have been removed (applying "remove-flags" policy)


Building phoneme dictionary: █████████████████████████████████████████████████████████████████████▓ 100.0%%


In [15]:
import json

with open(r'phoneme_cmn_dict.json', 'w+') as f:
    json.dump(phoneme_dict,f)


In [16]:
# 判断是否含有英文，以及转后字符长度限制
# import json
# phoneme_dict= json.load(open(r'phoneme_cmn_dict.json', 'r'))
i=0
for data in cmn_data:
    output_dir = data['trim_dir']
    metafile_path = os.path.join(data['name_dir'], data['output_metafile_name'])
    df = pd.read_excel(xlsx_list[i])
    i+=1
    with open(metafile_path, 'w', encoding='utf-8') as f:
        for index, row in df.iterrows(): 
            if bool(re.search('[a-zA-Z]', row.sentence_text)):
                continue
            if row.dataset_name == 'thchs30':speaker = row.speaker
            else:speaker = data['speaker']

            language = language
            audio_path = os.path.join(output_dir, row.voice_name)
            spectrogram_paths = "|"
            raw_text = to_pinyin(row.sentence_text)
#             raw_text = row.sentence_text

            if len(raw_text)>200:
                continue

            phonemized_text= _phonemize(raw_text,'cmn')
            print(f"{str(index).zfill(6)}|{speaker}|{language}|{audio_path}|{spectrogram_paths}|{row.sentence_text}|{phonemized_text}", file=f)
print('Done')

[WARNING] 1 utterances containing language switches on lines 2
[WARNING] extra phones may appear in the "cmn" phoneset
[WARNING] language switch flags have been removed (applying "remove-flags" policy)
[WARNING] 1 utterances containing language switches on lines 1
[WARNING] extra phones may appear in the "cmn" phoneset
[WARNING] language switch flags have been removed (applying "remove-flags" policy)
[WARNING] 1 utterances containing language switches on lines 1
[WARNING] extra phones may appear in the "cmn" phoneset
[WARNING] language switch flags have been removed (applying "remove-flags" policy)


Done


# 这里处理英文

In [5]:
sys.path.append('..')
from utils.text import to_phoneme
from utils.text import _phonemize

In [25]:
txt_file ='./Lance_Eng_CleanNoise_22k/Lance_Eng_CleanNoise_22k_trim33_new_val.txt'
txt_file_new='./re_gesw_7lang_self_clean22k/Lance_Eng_CleanNoise_22k_trim33_new_val.txt'
with open(txt_file, 'r', encoding='utf-8') as f:
    with open(txt_file_new, 'w', encoding='utf-8') as f2:
        
        for line in f:
            parts = line.strip().split('|')
            parts[1]="lance-en"
#             parts[7] = _phonemize(parts[6],'en-us')
            line_mod = '|'.join(parts)
            print(line_mod,file=f2)

In [121]:
name_dir = "Lance_Eng_CleanNoise_22k"
sub_dir = "Lance_CleanNoise"
trim_dir = "Lance_Eng_CleanNoise_22k_trim33"

speaker = "lance-en"
language = "en"

# name_dir = "Hotpot_Eng"
# sub_dir = "Hotpot_Eng"
# trim_dir = "Hotpot_Eng_22k_trim33"



In [122]:
xlsx_list = glob.glob("./{}/*.xlsx".format(name_dir))
xlsx_list

['./Lance_Eng_CleanNoise_22k/2Voice-Eng-Lance_CleanNoise22k.xlsx']

In [123]:
xlsx = xlsx_list[0]

In [124]:
df = pd.read_excel(xlsx)

In [125]:
df

,dataset_name,script_title,script_name,sentence_number,sentence_text,language,speaker,emotion,data_path,data_name,...,voice_format,voice_sr,prep_date_time,prep_path,prep_name,samples,frames,clean_date_time,cleaner_name,clean_text
0,Lance,Neurobiology_1,JPHE-001,0,Our mind is the most complicated piece of the ...,Eng,Lance,一般,\Lance_CleanNoise\CleanNoise-JPHE-001-04-01-01...,CleanNoise-JPHE-001-04-01-01-01_2.wav,...,wav,22050,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Lance,Neurobiology_1,JPHE-001,1,The rules that govern it are mysterious and el...,Eng,Lance,一般,\Lance_CleanNoise\CleanNoise-JPHE-001-04-01-01...,CleanNoise-JPHE-001-04-01-01-01_2.wav,...,wav,22050,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Lance,Neurobiology_1,JPHE-001,2,Maybe our brains just aren't complex enough to...,Eng,Lance,一般,\Lance_CleanNoise\CleanNoise-JPHE-001-04-01-01...,CleanNoise-JPHE-001-04-01-01-01_2.wav,...,wav,22050,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Lance,Neurobiology_1,JPHE-001,3,But that's not going to stop us from trying!,Eng,Lance,一般,\Lance_CleanNoise\CleanNoise-JPHE-001-04-01-01...,CleanNoise-JPHE-001-04-01-01-01_2.wav,...,wav,22050,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Lance,Neurobiology_1,JPHE-001,4,The word 'psychology' comes from the Latin for...,Eng,Lance,一般,\Lance_CleanNoise\CleanNoise-JPHE-001-04-01-01...,CleanNoise-JPHE-001-04-01-01-01_2.wav,...,wav,22050,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Lance,Neurobiology_1,JPHE-001,5,""" And while its formal definition has evolved ...",Eng,Lance,一般,\Lance_CleanNoise\CleanNoise-JPHE-001-04-01-01...,CleanNoise-JPHE-001-04-01-01-01_2.wav,...,wav,22050,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Lance,Neurobiology_1,JPHE-001,6,The term 'psychology' wasn't coined until arou...,Eng,Lance,一般,\Lance_CleanNoise\CleanNoise-JPHE-001-04-01-01...,CleanNoise-JPHE-001-04-01-01-01_2.wav,...,wav,22050,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Lance,Neurobiology_1,JPHE-001,7,"But of course, humans have always been curious...",Eng,Lance,一般,\Lance_CleanNoise\CleanNoise-JPHE-001-04-01-01...,CleanNoise-JPHE-001-04-01-01-01_2.wav,...,wav,22050,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Lance,Neurobiology_1,JPHE-001,8,Aristotle pondered the seed of human conscious...,Eng,Lance,一般,\Lance_CleanNoise\CleanNoise-JPHE-001-04-01-01...,CleanNoise-JPHE-001-04-01-01-01_2.wav,...,wav,22050,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Lance,Neurobiology_1,JPHE-001,8_b,As we have seen quite a lot here on our Crash ...,Eng,Lance,NaN,\Lance_CleanNoise\CleanNoise-JPHE-001-04-01-01...,CleanNoise-JPHE-001-04-01-01-01_2.wav,...,wav,22050,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [126]:
# lance selected 1
df = df[pd.isna(df.script_title)|
        (df.script_title == 'Your Brain Will Thank You for Being a Musician')|
        (df.script_title == 'Diluting blood plasma reverses aging')|
        (df.script_title == 'Roots of Mental Illness')|
        (df.script_title == 'Neurobiology_1')]

In [324]:
# hotpot selected 1
df = df[(df.dataset_name == 'EPwchina')|(df.dataset_name == 'Cspy')]

In [127]:
stft_window_ms = 50                  # size in ms of the Hann window of short-time Fourier transform, used during spectrogram computation
stft_shift_ms = 12.5 

In [128]:
import sys
sys.path.append("../")
from utils import audio

In [129]:
if not os.path.exists(os.path.join(name_dir, trim_dir)): os.makedirs(os.path.join(name_dir, trim_dir))
for index, row in df.iterrows(): 
    audio_path = os.path.join(name_dir, sub_dir, row.voice_name)
#     if resample:
#         sound = AudioSegment.from_file(audio_path)
#         sound_resample = sound.set_frame_rate(def_sample_rate).set_channels(1)
#         sound_resample.export(audio_path, format="wav")
    wav_data = audio.load(audio_path)
    wav_data_trim = audio.trim_silence(wav_data,stft_window_ms,stft_shift_ms,top_db=33)[0]
    audio_path_trim = os.path.join(name_dir, trim_dir, row.voice_name)
    if not os.path.exists(audio_path_trim):
        audio.save(wav_data_trim, audio_path_trim)

In [130]:
output_metafile_name = "Lance_Eng_CleanNoise_22k_trim33.txt"
# output_metafile_name = "gg_22k_trim33.txt"
# output_metafile_name = "cary_22k_trim33.txt"
# output_metafile_name = "Hotpot_Chn_22k_trim33.txt"
# output_metafile_name = "Lance_Eng_22k_trim33.txt"
# output_metafile_name = "Hotpot_Eng_22k_trim33.txt"

In [131]:
# 判断是否含有英文，以及转后字符长度限制
output_dir = trim_dir
metafile_path = os.path.join(name_dir, output_metafile_name)
with open(metafile_path, 'w', encoding='utf-8') as f:
    for index, row in df.iterrows(): 
#         if bool(re.search('[a-zA-Z]', row.sentence_text)):
#             continue
        speaker = row.dataset_name
#         speaker = speaker
        language = language
        audio_path = os.path.join(output_dir, row.voice_name)
        spectrogram_paths = "|"
        raw_text = row.sentence_text
        if len(raw_text)>200:
            continue
        phonemized_text= to_phoneme(raw_text,False,language)
        print(f'{str(index).zfill(6)}|{speaker}|{language}|{audio_path}|{spectrogram_paths}|{raw_text}|{phonemized_text}', file=f)

# 运行prepare_css_spectrograms.py

In [331]:
! ls

Hotpot_Chn    Lance_Eng     css10	make_data.ipynb
Hotpot_Eng    Mozilla	    css_comvoi	output.txt
LJSpeech-1.1  VCTK	    dump	prepare_css_spectrograms.py
Lance_Chn     cary_Chn	    gg_Chn	re_gesw_6lang
Lance_Chn_cl  comvoi_clean  input.txt	re_gesw_6lang_en


In [338]:
! ls cary_Chn

2Voice-Chn_cary_22k.xlsx  cary_22k  cary_22k_trim33  cary_22k_trim33.txt


In [17]:
# 得到修改后的metafile_path
# CUDA_VISIBLE_DEVICES=2 python3 prepare_css_spectrograms.py --data_directory Lance_Chn_cl  --data_txt Lance_Chn_22k_trim33_cl.txt
# !CUDA_VISIBLE_DEVICES=6 python3 prepare_css_spectrograms.py --data_directory Lance_Eng  --data_txt Lance_Eng_22k_trim33.txt
# CUDA_VISIBLE_DEVICES=2 python3 prepare_css_spectrograms.py --data_directory Hotpot_Chn  --data_txt Hotpot_Chn_22k_trim33.txt
# !CUDA_VISIBLE_DEVICES=6 python3 prepare_css_spectrograms.py --data_directory Hotpot_Eng  --data_txt Hotpot_Eng_22k_trim33.txt
# CUDA_VISIBLE_DEVICES=2 python3 prepare_css_spectrograms.py --data_directory gg_Chn  --data_txt gg_22k_trim33.txt
# CUDA_VISIBLE_DEVICES=2 python3 prepare_css_spectrograms.py --data_directory cary_Chn  --data_txt cary_22k_trim33.txt




!CUDA_VISIBLE_DEVICES=6 python3 prepare_css_spectrograms.py --data_directory gg_Chn_CleanNoise_22k  --data_txt gg_Chn_CleanNoise_22k_trim33.txt
# !CUDA_VISIBLE_DEVICES=6 python3 prepare_css_spectrograms.py --data_directory Mozilla  --data_txt ja_Mozilla.txt
!CUDA_VISIBLE_DEVICES=6 python3 prepare_css_spectrograms.py --data_directory Hotpot_Chn  --data_txt Hotpot_Chn_22k_trim33.txt
!CUDA_VISIBLE_DEVICES=6 python3 prepare_css_spectrograms.py --data_directory cary_Chn  --data_txt cary_22k_trim33.txt
!CUDA_VISIBLE_DEVICES=6 python3 prepare_css_spectrograms.py --data_directory Lance_Chn_CleanNoise_22k  --data_txt Lance_Chn_CleanNoise_22k_trim33.txt
!CUDA_VISIBLE_DEVICES=6 python3 prepare_css_spectrograms.py --data_directory thchs30_22k  --data_txt thchs30_22k_trim33.txt



Please wait, this may take a very long time.
Creating spectrograms for: gg_Chn_CleanNoise_22k_trim33.txt
Done.
Please wait, this may take a very long time.
Creating spectrograms for: Hotpot_Chn_22k_trim33.txt
Done.
Please wait, this may take a very long time.
Creating spectrograms for: cary_22k_trim33.txt
Done.
Please wait, this may take a very long time.
Creating spectrograms for: Lance_Chn_CleanNoise_22k_trim33.txt
Done.
Please wait, this may take a very long time.
Creating spectrograms for: thchs30_22k_trim33.txt
Done.


# 修改metafile内容生成train和val

## LJ和VCTK不需要运行prepare_css_spectrograms.py

In [190]:
# 自带提取代码
# name_dir = "LJSpeech-1.1"
# name_dir = "VCTK"

## 新加数据需要先运行prepare_css_spectrograms.py

In [78]:
# name_dir = "Lance_Chn_CleanNoise_22k"
# name_dir = "Mozilla"
# name_dir = "Lance_Eng_CleanNoise_22k"
# name_dir = "Hotpot_Chn"
# name_dir = "Hotpot_Eng"
# name_dir = "gg_Chn_CleanNoise_22k"
# name_dir = "cary_Chn"
name_dir = "thchs30_22k"

In [79]:
# single_speaker = True
# single_speaker = False

In [80]:
txt_list = glob.glob("{}/*.txt".format(name_dir))
txt_list

['thchs30_22k/thchs30_22k_trim33_train.txt',
 'thchs30_22k/thchs30_22k_trim33.txt',
 'thchs30_22k/thchs30_22k_trim33_val.txt']

In [81]:
txt_file = txt_list[1]
txt_file

'thchs30_22k/thchs30_22k_trim33.txt'

In [82]:
textlist=[]
with open(txt_file, encoding='utf-8') as f:
    for line in f:
        parts = line.strip().split('|')
#         if single_speaker:
#             parts[1] = parts[2]
#         parts[1] = "00-en"
#         parts[2] = "en"
        for i in range(3,6):
            parts[i] = "../{}/".format(name_dir)+ parts[i]
        line_mod = '|'.join(parts)+"\n"
        textlist.append(line_mod)

In [83]:
from sklearn.model_selection import train_test_split

In [84]:
train_list , val_list  = train_test_split(textlist,test_size=0.03, random_state=0)
len(textlist),len(train_list),len(val_list)

(13388, 12986, 402)

In [85]:
train_file = os.path.splitext(txt_file)[0]+"_train"+os.path.splitext(txt_file)[1]
val_file = os.path.splitext(txt_file)[0]+"_val"+os.path.splitext(txt_file)[1]

In [86]:
with open(train_file, 'w', encoding='utf-8') as f:
    for text in train_list:
        f.write(text)
with open(val_file, 'w', encoding='utf-8') as f:
    for text in val_list:
        f.write(text)

## 新建目标文件夹（训练主题）

In [87]:
# target_dir = "tran_Lance_Chn_LJ_Eng_10lang"
# target_dir = "tran_Lance_Chn_LJ_Eng"
# target_dir = "re_gesw_6lang_en"
target_dir = "re_gesw_6lang_22k_pinyin"
# target_dir = "wavernn_test"

In [88]:
if not os.path.exists(target_dir): os.makedirs(target_dir)

In [89]:
new_txt_file = os.path.join(target_dir, os.path.basename(train_file))
with open(new_txt_file, 'w', encoding='utf-8') as f:
    for text in train_list:
        f.write(text)
        
new_txt_file = os.path.join(target_dir, os.path.basename(val_file))
with open(new_txt_file, 'w', encoding='utf-8') as f:
    for text in val_list:
        f.write(text)

In [90]:
line_mod

'013387|D8|zh|../thchs30_22k/thchs30_22k_trim33/D8_999.wav|../thchs30_22k/spectrograms/013387.npy|../thchs30_22k/linear_spectrograms/013387.npy|国务委员兼国务院秘书长罗干、民政部部长多吉才让也一同前往延安看望人民群众。|kuo wu wuə iyɑ ə5n tɕiɑ ə5n kuo wu yɑ ə5n mi s.u ts.ɑ ŋ luo kɑ ə5n 、 mi nts.o ŋpu pu ts.ɑ ŋ tuo tɕi tshɑ iʐɑ ŋ jiɛ ji tho ŋ tɕhiɑ nwɑ ŋ jiɑ nɑ ə5n khɑ nwɑ ŋ ʐo nmi ə5n tɕhy nts.o ŋ 。\n'

In [91]:
txt_file

'thchs30_22k/thchs30_22k_trim33.txt'

In [92]:
new_txt_file

're_gesw_6lang_22k_pinyin/thchs30_22k_trim33_val.txt'

# 对其他data文件夹的txt增加“../dataname/”后复制到目标文件夹（不分训练测试集）

In [102]:
name_dir = "css10"
name_dir = "LJSpeech-1.1"
name_dir = "Lance_Chn"
txt_list = glob.glob("{}/*.txt".format(name_dir))
txt_list 

['Lance_Chn/Lance_Chn_22k_trim33_val.txt',
 'Lance_Chn/Lance_Chn_22k_trim33.txt',
 'Lance_Chn/Lance_Chn_22k_trim33_train.txt']

In [108]:
txt_file = txt_list[-1]

In [109]:
textlist=[]
with open(txt_file, encoding='utf-8') as f:
    for line in f:
        parts = line.strip().split('|')
        parts[1] = parts[2]   
        parts[1] = "lance"
        parts[2] = "zh"
#         for i in range(3,6):
#             parts[i] = "../{}/".format(name_dir)+ parts[i]
        line_mod = '|'.join(parts)+"\n"
        textlist.append(line_mod)
        
new_txt_file = os.path.join(target_dir, os.path.basename(txt_file))

with open(new_txt_file, 'w', encoding='utf-8') as f:
    for text in textlist:
        f.write(text)

In [170]:
line_mod

'009424|p250|en|../VCTK/wav48/p250/p250_496.wav|../VCTK/spectrograms/009424.npy|../VCTK/linear_spectrograms/009424.npy|Subsidy income will not be affected.|\n'

In [171]:
txt_file

'VCTK/vctk_mini.txt'

In [172]:
new_txt_file

're_gesw_6lang_en/vctk_mini_val.txt'

# 合并txt文件

In [93]:
target_dir = "re_gesw_6lang_22k_pinyin"

In [104]:
# txt_file_list_all = glob.glob("{}/*train*".format(target_dir))
txt_file_list_all = glob.glob("{}/*val*".format(target_dir))
txt_file_list_all

['re_gesw_6lang_22k_pinyin/Hotpot_Eng_22k_trim33_new_val.txt',
 're_gesw_6lang_22k_pinyin/val_ja_new.txt',
 're_gesw_6lang_22k_pinyin/ja_Mozilla_val-Copy1.txt',
 're_gesw_6lang_22k_pinyin/cary_22k_trim33_val.txt',
 're_gesw_6lang_22k_pinyin/Lance_Chn_CleanNoise_22k_trim33_val.txt',
 're_gesw_6lang_22k_pinyin/Lance_Eng_CleanNoise_22k_trim33_new_val.txt',
 're_gesw_6lang_22k_pinyin/gg_Chn_CleanNoise_22k_trim33_val.txt',
 're_gesw_6lang_22k_pinyin/vctk_mini_new_val.txt',
 're_gesw_6lang_22k_pinyin/lj_val.txt',
 're_gesw_6lang_22k_pinyin/val_5lang.txt',
 're_gesw_6lang_22k_pinyin/thchs30_22k_trim33_val.txt',
 're_gesw_6lang_22k_pinyin/Hotpot_Chn_22k_trim33_val.txt']

In [105]:
txt_file_list = sorted(txt_file_list_all)
txt_file_list

['re_gesw_6lang_22k_pinyin/Hotpot_Chn_22k_trim33_val.txt',
 're_gesw_6lang_22k_pinyin/Hotpot_Eng_22k_trim33_new_val.txt',
 're_gesw_6lang_22k_pinyin/Lance_Chn_CleanNoise_22k_trim33_val.txt',
 're_gesw_6lang_22k_pinyin/Lance_Eng_CleanNoise_22k_trim33_new_val.txt',
 're_gesw_6lang_22k_pinyin/cary_22k_trim33_val.txt',
 're_gesw_6lang_22k_pinyin/gg_Chn_CleanNoise_22k_trim33_val.txt',
 're_gesw_6lang_22k_pinyin/ja_Mozilla_val-Copy1.txt',
 're_gesw_6lang_22k_pinyin/lj_val.txt',
 're_gesw_6lang_22k_pinyin/thchs30_22k_trim33_val.txt',
 're_gesw_6lang_22k_pinyin/val_5lang.txt',
 're_gesw_6lang_22k_pinyin/val_ja_new.txt',
 're_gesw_6lang_22k_pinyin/vctk_mini_new_val.txt']

In [106]:
df_list = []
num = 0
for txt_file in txt_file_list:
    df = pd.read_csv(txt_file,sep='|',header=None)
    num += len(df)
    print(txt_file, "  ",len(df))
    df_list.append(df)
df_sum = pd.concat(df_list)
num

re_gesw_6lang_22k_pinyin/Hotpot_Chn_22k_trim33_val.txt    9
re_gesw_6lang_22k_pinyin/Hotpot_Eng_22k_trim33_new_val.txt    14
re_gesw_6lang_22k_pinyin/Lance_Chn_CleanNoise_22k_trim33_val.txt    24
re_gesw_6lang_22k_pinyin/Lance_Eng_CleanNoise_22k_trim33_new_val.txt    12
re_gesw_6lang_22k_pinyin/cary_22k_trim33_val.txt    10
re_gesw_6lang_22k_pinyin/gg_Chn_CleanNoise_22k_trim33_val.txt    18
re_gesw_6lang_22k_pinyin/ja_Mozilla_val-Copy1.txt    335
re_gesw_6lang_22k_pinyin/lj_val.txt    393
re_gesw_6lang_22k_pinyin/thchs30_22k_trim33_val.txt    402
re_gesw_6lang_22k_pinyin/val_5lang.txt    632
re_gesw_6lang_22k_pinyin/val_ja_new.txt    64
re_gesw_6lang_22k_pinyin/vctk_mini_new_val.txt    283


2196

In [107]:
df_sum

,0,1,2,3,4,5,6,7
0,112,hotpot-zh,zh,../Hotpot_Chn/Hotpot_Chn_22k_trim33/Hotpot_Chn...,../Hotpot_Chn/spectrograms/000112.npy,../Hotpot_Chn/linear_spectrograms/000112.npy,另一件要注意的是,li ŋ ji tɕiɑ ə5n jiɑ ou5 ts.u ji tə s.i.
1,176,hotpot-zh,zh,../Hotpot_Chn/Hotpot_Chn_22k_trim33/Hotpot_Chn...,../Hotpot_Chn/spectrograms/000176.npy,../Hotpot_Chn/linear_spectrograms/000176.npy,暴露你们自己吧，骗子们,pɑ olu ni mon tsi̪ tɕi pɑ ， phiɑ ntsi̪ mon
2,12,hotpot-zh,zh,../Hotpot_Chn/Hotpot_Chn_22k_trim33/Hotpot_Chn...,../Hotpot_Chn/spectrograms/000012.npy,../Hotpot_Chn/linear_spectrograms/000012.npy,有时，我安排定时发布视频,jio us.i. ， wo ei5 nphɑ ai5 ti ŋs.i. fɑ pu s.i...
3,220,hotpot-zh,zh,../Hotpot_Chn/Hotpot_Chn_22k_trim33/Hotpot_Chn...,../Hotpot_Chn/spectrograms/000220.npy,../Hotpot_Chn/linear_spectrograms/000220.npy,在那之前我已经好多年没哭了,tsɑ ai5 nɑ ts.i. tɕhiɑ ə5n wo ji tɕi ŋ xɑ otuo...
4,206,hotpot-zh,zh,../Hotpot_Chn/Hotpot_Chn_22k_trim33/Hotpot_Chn...,../Hotpot_Chn/spectrograms/000206.npy,../Hotpot_Chn/linear_spectrograms/000206.npy,我们将做出巨大的改变,wo mon tɕiɑ ŋ tsuo ts.hu tɕy tɑ tə kɑ ipiɑ ə5n
...,...,...,...,...,...,...,...,...
278,7410,p246,en,../VCTK/wav48/p246/p246_011.wav,../VCTK/spectrograms/007410.npy,../VCTK/linear_spectrograms/007410.npy,When a man looks for something beyond his reac...,"wɛn ɐ mæn lʊks fɔːɹ sʌmθɪŋ bᵻjɔnd hɪz ɹiːtʃ , ..."
279,7240,p245,en,../VCTK/wav48/p245/p245_199.wav,../VCTK/spectrograms/007240.npy,../VCTK/linear_spectrograms/007240.npy,The second is the reaction of the Hibs fans.,ðə sɛkənd ɪz ðə ɹɪækʃən ʌvðə hɪbz fænz .
280,3081,p233,en,../VCTK/wav48/p233/p233_103.wav,../VCTK/spectrograms/003081.npy,../VCTK/linear_spectrograms/003081.npy,Police said the family was helping with the se...,pəliːs sɛd ðə fæmɪli wʌz hɛlpɪŋ wɪððə sɜːtʃ .
281,5218,p239,en,../VCTK/wav48/p239/p239_219.wav,../VCTK/spectrograms/005218.npy,../VCTK/linear_spectrograms/005218.npy,So I got a blood test.,soʊ aɪ ɡɑːt ɐ blʌd tɛst .


In [108]:
# output_metafile_name = "train.txt"
output_metafile_name = "val.txt"
metafile_path = os.path.join(target_dir, output_metafile_name)
with open(metafile_path, 'w', encoding='utf-8') as f:
    for txt_file in txt_file_list:
        with open(txt_file, encoding='utf-8') as f2:
            for line in f2:
                # print(f'{line}', file=f)
                f.write(line)

In [100]:
sorted(["f","h","a"]).index("a")

0

In [101]:
["f","h","a"].index("a")

2

In [102]:
metafile_path

're_gesw_6lang_22k_pinyin/train.txt'